<a href="https://colab.research.google.com/github/Austfi/SNOWPACKforPatrollers/blob/main/SNOWPACKforPatrollers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SNOWPACK Model Setup and Execution

This notebook guides you through the process of setting up and running the SNOWPACK model. It includes steps for installing necessary libraries, compiling the SNOWPACK and MeteoIO code, configuring the model, fetching meteorological data from historical weather models, and running SNOWPACK.

# The following cells below setup SNOWPACK AND MeteoIO and the PATH strcuture to run it. They should not be edited. Pressing the play button below will run them all.

In [ ]:
# @title Install environment updates
!apt-get update
!apt-get install -y build-essential cmake git liblapack-dev numdiff

%pip install openmeteo-requests requests-cache retry-requests pandas numpy

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from pathlib import Path
import openmeteo_requests
import requests_cache
from retry_requests import retry
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,799 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Fetched 5,780 kB in 3s

In [ ]:
# @title Download SNOWPACK
!git clone https://github.com/snowpack-model/snowpack.git


Cloning into 'snowpack'...
remote: Enumerating objects: 15270, done.
remote: Counting objects: 100% (989/989), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 15270 (delta 745), reused 733 (delta 660), pack-reused 14281 (from 2)
Receiving objects: 100% (15270/15270), 84.87 MiB | 13.74 MiB/s, done.
Resolving deltas: 100% (10662/10662), done.
Updating files: 100% (1756/1756), done.


In [ ]:
# @title Make  ~/usr folder
!mkdir ~/usr/

In [ ]:
# @title Compile MeteoIO  (Runs about 6 Mins)
!cd /content/snowpack/Source/meteoio/ && \
cmake -S . -B build-meteoio -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=$HOME/usr && \
cmake --build build-meteoio -j2 && \
cmake --install build-meteoio

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test HAVE_STRUCT_STAT_ST_FLAGS
-- Performing Test HAVE_STRUCT_STAT_ST_FLAGS - Failed
-- Performing Test HAVE_FEENABLE
-- Performing Test HAVE_FEENABLE - Success
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (3.2s)
-- Generating done (0.2s)
-- Build files have been written to: /content/snowpack/Source/meteoio/build-meteoio
[  1%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ct

In [ ]:
# @title Compile SNOWPACK (Runs about 2 mins)

!cd /content/snowpack/Source/snowpack/ && \
cmake -S . -B build-snowpack -DCMAKE_BUILD_TYPE=Release \
      -DCMAKE_PREFIX_PATH=$HOME/usr \
      -DDEBUG_ARITHM=OFF \
      -DENABLE_LAPACK=ON \
      -DMETEOIO_LIBRARY=$HOME/usr/lib/libmeteoio.so && \
cmake --build build-snowpack -j2 && \
cmake --install build-snowpack

In [ ]:
# @title Set up PATH to compiled code
!PATH="/home/app/usr/bin:${PATH}"
!LD_LIBRARY_PATH="/home/app/usr/lib:${LD_LIBRARY_PATH}"
!which snowpack

# The above cells have setup up SNOWPACK and made it ready to run. The below cells create the files, collect data, and run the snowpack model.

In [ ]:
#@title SNOWPACK Configuration Files Generation
"""
Generate .sno and .ini configuration files for SNOWPACK simulations
"""

from typing import List, Tuple, Optional
import os

#@markdown ## Station Configuration
station_display_name = "keystone alpine"  #@param {"type":"string"}
station_slug_input = "keystone_model"  #@param {"type":"string"}
latitude = 39.56858687967004  #@param {"type":"number"}
longitude = -105.91900397453021  #@param {"type":"number"}
altitude = 3614  #@param {"type":"integer"}
timezone = -7  #@param {"type":"number"}
profile_date = "2024-11-01T00:00:00"  #@param {"type":"string"}

#@markdown ## Virtual Slopes Configuration
num_slopes = 5  #@param {"type":"number", "min":1, "max":10}
include_flat = True  #@param {"type":"boolean"}
default_slope_angle = 38.0  #@param {"type":"number", "min":0, "max":90}

#@markdown ## Slope Directions (degrees: 0=North, 90=East, 180=South, 270=West)
north_slope = True  #@param {"type":"boolean"}
east_slope = True  #@param {"type":"boolean"}
south_slope = True  #@param {"type":"boolean"}
west_slope = True  #@param {"type":"boolean"}
custom_directions = ""  #@param {"type":"string"} {description:"Comma-separated azimuth angles (e.g., 45,135,225,315)"}

#@markdown ## SNOWPACK Settings
meas_tss = "false"  #@param ["true","false"]
enforce_measured_snow_heights = "false"  #@param ["true","false"]

#@markdown ## Output Configuration
sno_directory = "/content/snowpack/Source/snowpack/tests/input"  #@param {"type":"string"}
ini_directory = "/content/snowpack/Source/snowpack/tests/keystone"  #@param {"type":"string"}
generate_config_files = True  #@param {"type":"boolean"}

COLAB_ROOT = "/content"
IS_COLAB = os.path.exists(COLAB_ROOT)


def _slugify(value: str) -> str:
    cleaned = [ch.lower() if ch.isalnum() else "_" for ch in value.strip()]
    slug = "".join(cleaned)
    parts = [part for part in slug.split("_") if part]
    return "_".join(parts)


station_slug = _slugify(station_slug_input) if station_slug_input.strip() else _slugify(station_display_name)
if not station_slug:
    station_slug = "station"
station_name = station_display_name.strip() or station_slug
station_id = station_slug  # backwards compatibility for downstream cells


def _normalize_path(value: str) -> str:
    expanded = os.path.expanduser(value)
    if IS_COLAB:
        return expanded
    return os.path.abspath(expanded)


sno_directory = _normalize_path(sno_directory)
ini_directory = _normalize_path(ini_directory)
paths = {
    "sno": sno_directory,
    "ini": ini_directory,
    "smet": sno_directory,
}


def build_station_identifier(model: Optional[str] = None) -> str:
    return f"{station_slug}_{model}" if model else station_slug


def generate_slopes(include_flat: bool,
                    north_slope: bool,
                    east_slope: bool,
                    south_slope: bool,
                    west_slope: bool,
                    custom_directions: str,
                    default_slope_angle: float) -> List[Tuple[float, float]]:
    """Generate list of virtual slopes based on user selections."""
    slopes: List[Tuple[float, float]] = []

    if include_flat:
        slopes.append((0.0, 0.0))

    if north_slope:
        slopes.append((default_slope_angle, 0.0))
    if east_slope:
        slopes.append((default_slope_angle, 90.0))
    if south_slope:
        slopes.append((default_slope_angle, 180.0))
    if west_slope:
        slopes.append((default_slope_angle, 270.0))

    if custom_directions.strip():
        try:
            custom_angles = [float(x.strip()) for x in custom_directions.split(',')]
            for angle in custom_angles:
                if 0 <= angle <= 360:
                    slopes.append((default_slope_angle, angle))
        except ValueError:
            print("Warning: Invalid custom directions format. Use comma-separated numbers.")

    return slopes


def build_slope_filename(index: int) -> str:
    """Return slope filename following <station_slug><index>.sno convention."""
    return f"{station_slug}{index + 1}.sno"


def create_sno_content(station_id: str,
                       station_name: str,
                       longitude: float,
                       latitude: float,
                       altitude: float,
                       timezone: float,
                       profile_date: str,
                       slope_angle: float,
                       slope_azimuth: float) -> str:
    """Create .sno file content for a single slope."""

    content = f"""SMET 1.1 ASCII
[HEADER]
station_id       = {station_id}
station_name     = {station_name}
longitude        = {longitude}
latitude         = {latitude}
altitude         = {altitude}
nodata           = -999
tz               = {timezone}
source           = AWSOME
prototype        = SNOWPACK
ProfileDate      = {profile_date}
HS_Last          = 0.0000
SlopeAngle       = {slope_angle}
SlopeAzi         = {slope_azimuth}
nSoilLayerData   = 0
nSnowLayerData   = 0
SoilAlbedo       = 0.09
BareSoil_z0      = 0.020
CanopyHeight     = 0.00
CanopyLeafAreaIndex = 0.00
CanopyDirectThroughfall = 1.00
ErosionLevel     = 0
TimeCountDeltaHS = 0.000000
WindScalingFactor = 1.00

fields           = timestamp Layer_Thick  T  Vol_Frac_I  Vol_Frac_W  Vol_Frac_V  Vol_Frac_S Rho_S Conduc_S HeatCapac_S  rg  rb  dd  sp  mk mass_hoar ne CDot metamo
[DATA]
"""

    return content


def create_ini_content(station_identifier: str,
                       meteo_filename: str,
                       snowfiles: List[str],
                       meas_tss: str,
                       enforce_measured_snow_heights: str) -> str:
    """Create .ini file content with multiple SNOWFILE entries."""

    content = f"""[General]
BUFFER_SIZE = 370
BUFF_BEFORE = 1.5

[Input]
COORDSYS = UTM
COORDPARAM = 13S
TIME_ZONE = -7

METEO = SMET
METEOPATH = ../input
METEOFILE1 = {meteo_filename}
"""

    for i, snowfile in enumerate(snowfiles, 1):
        content += f"SNOWFILE{i} = {snowfile}
"

    content += f"""
[Output]
METEOPATH = ../output
SNOWFILE = true
PROFOUT = TRUE
PROFOUTINTERVAL = 86400

default::WRITE_PROCESSED_METEO = TRUE
PACK_TIMESTEP = 900

[SNOWPACK]
DNSN_STOP = FALSE
MEAS_TSS = {meas_tss.upper()}
ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights.upper()}
NUMBER_SLOPES = {len(snowfiles)}

[Filters]
SNOWPACK::filters = snowpack

[snowpack]
filter1 = HNW_VS_RH

[FilterParameters]
HNW_VS_RH::max = 0.9
HNW_VS_RH::nhour = 2
HNW_VS_RH::min = 0.1

[SnowpackAdvanced]
SNOWPACK::TIME_STEP = 900
SNOWPACK::ITERATIONS = 15

[Stress]
stress::value = 50

[SnowpackSolver]
SNOWPACKADVANCED::CSTOP = 0.1

[Parameters]
HN_DENSITY = 70
HN_DENSITY_FIXEDVALUES = FALSE
HN_DENSITY_TYPE = HET
HN_DENSITY_COEFF = 3.28
HN_DENSITY_COEFF2 = 19.6
HN_V = 0.0015
HN_V_OLDVERSION = FALSE
SW_MODE = FAST
CANOPY = FALSE
THRESH_SW = 50
ALPINE3D = FALSE
ILWR_MODEL = Both
ALBEDO_MODEL = VARIANT_HM
ROUGHNESS_LENGTH = 0.002
STOP_PROG = 0
SEA = FALSE
RESET_RHO = FALSE
SNP_SEA = FALSE
ATM_CONTROLS = FALSE
OLDTAU = FALSE
AVG_PAN = TRUE
SSFLAGS = NOT_RELIABLE REFERENCE
WG = FALSE
SUBL = FALSE
TRANSITION_GRAIN_METAMORPHISM = TRUE
CLOUD = TRUE

[Filters2]
SNOWPACK::filters2 = snowpack2

[snowpack2]
filter1 = RH
filter2 = VW
filter3 = DW

[FilterParameters2]
RH::duration = 3600
RH::operation = OR
RH::min = 0.0
RH::max = 1.2
VW::duration = 3600
VW::operation = OR
VW::min = 0.0
VW::max = 50.0
DW::duration = 3600
DW::operation = OR
DW::min = 0.0
DW::max = 360.0

[Filters3]
SNOWPACK::filters3 = snowpack3

[snowpack3]
filter1 = ILWR
filter2 = ISWR
filter3 = RH
filter4 = VW
filter5 = DW

[FilterParameters3]
ILWR::arg1::type = local
ILWR::arg1::name = IPR
ISWR::arg1::type = local
ISWR::arg1::name = ISR
RH::arg1::type = local
RH::arg1::name = RH
VW::arg1::type = local
VW::arg1::name = VW
DW::arg1::type = local
DW::arg1::name = DW

[Interpolations]
ILWR::resample1 = nearest
ILWR::arg1::extrapolate = true
ILWR::arg1::nearest = 3600
ISWR::resample1 = nearest
ISWR::arg1::extrapolate = true
ISWR::arg1::nearest = 3600
TA::resample1 = linear
TA::arg1::extrapolate = true
TA::arg1::nearest = 3600
TA::resample2 = linear
PSUM::resample1 = linear
PSUM::arg1::extrapolate = false
TSG::resample1 = linear
TSG::arg1::nearest = 3600
TSW::resample1 = linear
TSW::arg1::nearest = 3600
VW::resample1 = nearest
VW::arg1::extrapolate = true
DW::resample1 = nearest
DW::arg1::extrapolate = true
RH::resample1 = nearest
RH::arg1::extrapolate = true

[Interpolations2]
ILWR::arg1::nearest = 3600
ISWR::arg1::nearest = 3600
RH::arg1::nearest = 3600
VW::arg1::nearest = 3600
DW::arg1::nearest = 3600
ILWR::arg1::max = 600
ISWR::arg1::max = 600
RH::arg1::max = 600
VW::arg1::max = 600
DW::arg1::max = 600

[Interpolations3]
ILWR::arg1::min = 0.0
ILWR::arg1::max = 450.0
ISWR::arg1::min = 0.0
ISWR::arg1::max = 1400.0
RH::arg1::min = 0.0
RH::arg1::max = 1.0
VW::arg1::min = 0.0
VW::arg1::max = 50.0

[Interpolations1D]
MAX_GAP_SIZE = 86400
PSUM::resample1 = accumulate
PSUM::ARG1::period = 900
HS::resample1 = linear
HS::ARG1::MAX_GAP_SIZE = 43200
VW::resample1 = nearest
VW::ARG1::extrapolate = true
DW::resample1 = nearest
DW::ARG1::extrapolate = true

[Generators]
ILWR::generator1 = AllSky_LW
ILWR::arg1::type = Carmona
ILWR::arg1::shade_from_dem = FALSE
ILWR::arg1::use_rswr = FALSE
ILWR::generator2 = ClearSky_LW
ILWR::arg2::type = Dilley
"""

    return content


def main() -> None:
    """Main execution function."""

    station_identifier = build_station_identifier()

    print("SNOWPACK Configuration Files Generation")
    print("=" * 50)
    print(f"Station display name: {station_name}")
    print(f"Station identifier : {station_identifier}")
    print(f"Location: {latitude:.6f}°N, {longitude:.6f}°W, {altitude}m")
    print(f"SNO directory: {paths['sno']}")
    print(f"INI directory: {paths['ini']}")
    print("=" * 50)

    if not generate_config_files:
        print("Configuration generation disabled. Set 'generate_config_files' to True to generate files.")
        return

    os.makedirs(paths['sno'], exist_ok=True)
    os.makedirs(paths['ini'], exist_ok=True)

    slopes = generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle)

    if len(slopes) > num_slopes:
        slopes = slopes[:num_slopes]
        print(f"Warning: Limited to {num_slopes} slopes as requested")

    snowfiles: List[str] = []
    sno_files_created = []

    for i, (angle, azimuth) in enumerate(slopes):
        sno_filename = build_slope_filename(i)
        snowfiles.append(sno_filename)

        sno_content = create_sno_content(
            station_id=station_identifier,
            station_name=station_name,
            longitude=longitude,
            latitude=latitude,
            altitude=altitude,
            timezone=timezone,
            profile_date=profile_date,
            slope_angle=angle,
            slope_azimuth=azimuth,
        )

        sno_filepath = os.path.join(paths['sno'], sno_filename)
        with open(sno_filepath, "w") as f:
            f.write(sno_content)

        sno_files_created.append(sno_filepath)

    meteo_filename = f"{station_slug}.smet"
    ini_content = create_ini_content(station_identifier, meteo_filename, snowfiles, meas_tss, enforce_measured_snow_heights)

    ini_filename = os.path.join(paths['ini'], f"{station_slug}.ini")
    with open(ini_filename, "w") as f:
        f.write(ini_content)

    print("
Configuration files created:")
    print(f"   .ini file: {ini_filename}")
    print("   .sno files:")
    for sno_file in sno_files_created:
        print(f"     {sno_file}")

    print(f"
Virtual slopes configured: {len(slopes)}")
    for i, (angle, azimuth) in enumerate(slopes):
        sno_filename = build_slope_filename(i)
        if angle == 0.0:
            descriptor = "Flat (0°)"
        else:
            if azimuth == 0.0:
                direction = "North"
            elif azimuth == 90.0:
                direction = "East"
            elif azimuth == 180.0:
                direction = "South"
            elif azimuth == 270.0:
                direction = "West"
            else:
                direction = f"{azimuth}°"
            descriptor = f"{angle}° slope facing {direction}"
        print(f"   Slope {i + 1}: {descriptor} -> {sno_filename}")

    print("
SNOWPACK settings:")
    print(f"   MEAS_TSS = {meas_tss}")
    print(f"   ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights}")
    print(f"   NUMBER_SLOPES = {len(slopes)}")

    print("
🎉 SNOWPACK configuration generation complete!")


if __name__ == "__main__":
    main()



In [ ]:
# @title SMET Handling and API Setup
def create_smet_from_weather_data(weather_df, output_path, station_id, station_name, latitude, longitude, altitude):
    """
    Create SMET file from weather DataFrame
    """
    # Ensure timestamp is datetime
    if 'timestamp' in weather_df.columns:
        weather_df = weather_df.copy()
        weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
    else:
        weather_df = weather_df.copy()
        weather_df.reset_index(inplace=True)
        weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])

    # Define fields in order
    fields = ['timestamp', 'TA', 'RH', 'TSG', 'VW', 'DW', 'ISWR', 'PSUM']
    if 'HS' in weather_df.columns:
        fields.append('HS')

    # Create output directory if needed
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(output_path, 'w') as f:
        # Write SMET header
        f.write("SMET 1.1 ASCII\n")
        f.write("[HEADER]\n")
        f.write(f"station_id = {station_id}\n")
        f.write(f"station_name = {station_name}\n")
        f.write(f"latitude = {latitude:.10f}\n")
        f.write(f"longitude = {longitude:.10f}\n")
        f.write(f"altitude = {altitude}\n")
        f.write("nodata = -777\n")
        f.write("Tz = -7\n")
        f.write(f"fields = {'\t'.join(fields)}\n")

        # Units
        field_count = len(fields)
        units_offset = ["0"] * field_count
        units_multiplier = ["1"] * field_count
        f.write(f"units_offset     = {' '.join(units_offset)}\n")
        f.write(f"units_multiplier = {' '.join(units_multiplier)}\n")

        # Data section
        f.write("[DATA]\n")

        # Write data rows
        for _, row in weather_df.iterrows():
            values = []
            for field in fields:
                if field == 'timestamp':
                    values.append(row[field].strftime('%Y-%m-%dT%H:%M:%S'))
                else:
                    val = row.get(field, -777)
                    if pd.isna(val):
                        val = -777
                    values.append(f"{val:.2f}")
            f.write("\t".join(values) + "\n")

def fetch_openmeteo_historical(lat, lon, start_date, end_date, model="gfs_hrrr", elevation=None):
    """
    Fetch historical forecast data from Open-Meteo API
    """
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "wind_direction_10m",
            "snow_depth",
            "direct_radiation",
            "precipitation"
        ],
        "models": model,
        "wind_speed_unit": "ms",
    }

    if elevation is not None:
        params["elevation"] = elevation

    print(f"Fetching {model} data for {lat}, {lon} from {start_date} to {end_date}")

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
        print(f"Elevation: {response.Elevation()} m asl")

        # Process hourly data
        hourly = response.Hourly()
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
        hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
        hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
        hourly_wind_direction_10m = hourly.Variables(3).ValuesAsNumpy()
        hourly_snow_depth = hourly.Variables(4).ValuesAsNumpy()
        hourly_direct_radiation = hourly.Variables(5).ValuesAsNumpy()
        hourly_precipitation = hourly.Variables(6).ValuesAsNumpy()

        # Create time index
        time_index = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )

        # Create DataFrame
        df = pd.DataFrame({
            'timestamp': time_index,
            'TA': hourly_temperature_2m + 273.15,  # Convert to Kelvin
            'RH': hourly_relative_humidity_2m / 100.0,  # Convert to fraction
            'VW': hourly_wind_speed_10m,  # Already in m/s
            'DW': hourly_wind_direction_10m,  # Already in degrees
            'HS': hourly_snow_depth,  # Snow depth in meters
            'ISWR': hourly_direct_radiation,  # Direct radiation in W/m²
            'PSUM': hourly_precipitation,  # Precipitation in mm
        })

        # Add missing fields
        df['TSG'] = 273.15  # Ground temperature (assumed)

        # Handle missing values
        df = df.replace([np.inf, -np.inf], np.nan)

        print(f"Fetched {len(df)} records")
        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None


In [ ]:
# @title SMET Generation Parameters
# @markdown Configure your parameters below and run this cell to generate SMET files

# @markdown Time Period
start_date = "2024-11-01" # @param {type:"date"}
end_date = "2025-04-30" # @param {type:"date"}

# @markdown Weather Models
model_selection = "ifs" # @param ["hrrr","nbm","ifs"]

# @markdown Generate SMET Files
generate_files = True # @param {type:"boolean"}

print("SMET Generation Parameters")
print("=" * 50)
print(f"Station display name: {station_name}")
print(f"Station identifier : {build_station_identifier()}")
print(f"Location: {latitude}, {longitude} (altitude: {altitude}m)")
print(f"Period: {start_date} to {end_date}")
print(f"Model: {model_selection}")
print(f"Generate: {generate_files}")
print(f"SMET output directory: {paths['smet']}")
print("=" * 50)



In [ ]:
# @title Generate SMET Files
# @markdown Run this cell to generate SMET files with the parameters above

import os
from pathlib import Path

output_directory = paths["smet"]

model_mapping = {
    "hrrr": "gfs_hrrr",
    "nbm": "ncep_nbm_conus",
    "ifs": "ecmwf_ifs"
}

if generate_files:
    print("Starting SMET generation...")

    Path(output_directory).mkdir(parents=True, exist_ok=True)

    selected_models = [model_selection]

    valid_models = ['hrrr', 'nbm', 'nam', 'graphcast', 'ifs']
    selected_models = [model for model in selected_models if model in valid_models]

    if not selected_models:
        print("No valid models selected!")
        print(f"Available models: {', '.join(valid_models)}")
    else:
        print(f"Processing {len(selected_models)} model(s): {', '.join(selected_models)}")
        print("="*50)

        successful_files = []

        for model in selected_models:
            print(f"
Processing {model}...")

            openmeteo_model = model_mapping[model]

            df = fetch_openmeteo_historical(
                lat=latitude,
                lon=longitude,
                start_date=start_date,
                end_date=end_date,
                model=openmeteo_model,
                elevation=altitude
            )

            if df is None or df.empty:
                print(f"Failed to fetch data for {model}")
                continue

            station_identifier = build_station_identifier(model)
            station_header_name = station_name
            output_file = os.path.join(output_directory, f"{station_slug}.smet")

            try:
                create_smet_from_weather_data(
                    weather_df=df,
                    output_path=output_file,
                    station_id=station_identifier,
                    station_name=station_header_name,
                    latitude=latitude,
                    longitude=longitude,
                    altitude=altitude
                )

                print(f"SMET file created: {output_file}")
                print(f"Records: {len(df)}")
                print(f"Period: {df['timestamp'].min()} to {df['timestamp'].max()}")

                successful_files.append(output_file)

            except Exception as e:
                print(f"Error creating SMET for {model}: {e}")
                continue

        print(f"
Successfully generated {len(successful_files)} SMET files:")
        for file in successful_files:
            print(f"  {file}")

        if successful_files:
            print(f"
Files saved to: {output_directory}")
else:
    print("Generation disabled. Set 'generate_files' to True to generate SMET files.")



In [ ]:
# @title Run this cell to run SNOWPACK
!cd /content/snowpack/Source/snowpack/tests/keystone/ && \
/usr/local/bin/snowpack -c /content/snowpack/Source/snowpack/tests/keystone/keystone_model.ini -e 2025-04-01T00:00


In [ ]:
# @title Download .pro files and Open niViz
# @markdown # 5 files will be download.
# @markdown # 1 = flat, 2=N, 3=E, 4=S, 5=W
import glob, shutil, os
from IPython.display import FileLink, Markdown
from google.colab import files

# Locate all .pro files
pro_files = glob.glob('/content/snowpack/Source/snowpack/tests/keystone/output/*.pro')

if pro_files:
    # Copy all files to a simple location
    downloaded_files = []
    for pro_file in pro_files:
        filename = pro_file.split('/')[-1]
        shutil.copy(pro_file, f'/content/{filename}')
        downloaded_files.append(filename)

    # Create a zip file with all .pro files
    import zipfile
    zip_filename = 'snowpack_profiles.zip'
    with zipfile.ZipFile(f'/content/{zip_filename}', 'w') as zipf:
        for filename in downloaded_files:
            zipf.write(f'/content/{filename}', filename)

    # Download the zip file
    files.download(f'/content/{zip_filename}')

    # Show how to open in niViz
    display(Markdown(f"""
    ---
    {len(downloaded_files)} SNOWPACK profile files have been downloaded as **`{zip_filename}`**.

    Next Step — View in niViz
    1. Go to https://run.niviz.org
    2. Click "File" → "Open Profile" or drag any of the downloaded files into the screen
    3. Select any of the downloaded files:
    """))

    for filename in downloaded_files:
        display(Markdown(f"   - **`{filename}`**"))

else:
    display(Markdown("No .pro files found in the output directory."))